In [1]:
import subprocess
import os
import shlex
import logging
import shutil

log = logging.getLogger(__name__)
logging.basicConfig(level=logging.DEBUG)

WORK_DIR = 'work'
HELPDOC_FILES = ['dev-tools/helpdoc', 'dev-tools/helpdoc.d', 'dev-tools/helpdoc.schema', 'dev-tools/input_xx.xsl', 'GUI/Guib/lib']
DEFS_TO_PARSE = ['PW/Doc/INPUT_PW.def', 'PP/Doc/INPUT_PROJWFC.def']
VERSION = ['6.3', '6.8', '7.0', '7.2']
DATABSE_DIR = 'database/espresso'

In [2]:
def run_command(command):
    log.debug(f"Command: {command}")
    command = shlex.split(command)
    result = subprocess.run(command, capture_output=True, check=True)
    if result.stdout:
        log.debug(f"Command stdout: {result.stdout.decode('utf-8')}")
    if result.stderr:
        log.debug(f"Command stderr: {result.stderr.decode('utf-8')}")
    return result

In [3]:
# Create work directory and go there
root = os.getcwd()
work_dir = os.path.join(root, WORK_DIR)
# TODO: this is temporary
if os.path.exists(work_dir):
    shutil.rmtree(work_dir)
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
os.chdir(work_dir)

# Commands to set up minimal helpdoc environment
qe_dir = os.path.join(work_dir, 'q-e')
cmd_clone = "git clone --filter=blob:none --sparse https://gitlab.com/QEF/q-e.git"
run_command(cmd_clone)
os.chdir(qe_dir)
cmd_fetch_tags = "git fetch --all --tags"
run_command(cmd_fetch_tags)

cmd_checkout_files = ["git sparse-checkout add"]
cmd_checkout_files = " ".join(cmd_checkout_files + HELPDOC_FILES + DEFS_TO_PARSE)
run_command(cmd_checkout_files)

# Commands for picking the right version
devtools_dir = os.path.join(qe_dir, 'dev-tools')
for v in VERSION:
    tag = v
    tag += "MaX" if v in ("6.3", "6.5") else ""
    tag += "MaX-Release" if v == "6.7" else ""
    cmd_checkout_tag = f"git checkout tags/qe-{tag} -b qe-{tag}"
    run_command(cmd_checkout_tag)
    database_dir = os.path.join(root, DATABSE_DIR, v)
    if not os.path.exists(database_dir):
        os.makedirs(database_dir)

    files = [os.path.join(qe_dir, def_file) for def_file in DEFS_TO_PARSE]
    for def_file in files:
        dir = os.path.dirname(def_file)
        cmd_link_xsl = f"ln -sf {devtools_dir}/input_xx.xsl {dir}/input_xx.xsl"
        run_command(cmd_link_xsl)
        cmd_helpdoc = f"{devtools_dir}/helpdoc --version {v} {def_file}"
        run_command(cmd_helpdoc)

        # Copy the generated files to the database directory using os module
        xml_file = os.path.splitext(def_file)[0] + '.xml'
        html_file = os.path.splitext(def_file)[0] + '.html'
        # Explicit destination is needed to overwrite existing files
        shutil.move(html_file, os.path.join(database_dir, os.path.basename(html_file)))
        shutil.move(xml_file, os.path.join(database_dir, os.path.basename(xml_file)))

os.chdir(root)

DEBUG:__main__:Command: git clone --filter=blob:none --sparse https://gitlab.com/QEF/q-e.git
DEBUG:__main__:Command stderr: Cloning into 'q-e'...

DEBUG:__main__:Command: git fetch --all --tags
DEBUG:__main__:Command stdout: Fetching origin

DEBUG:__main__:Command: git sparse-checkout add dev-tools/helpdoc dev-tools/helpdoc.d dev-tools/helpdoc.schema dev-tools/input_xx.xsl GUI/Guib/lib PW/Doc/INPUT_PW.def PP/Doc/INPUT_PROJWFC.def
DEBUG:__main__:Command: git checkout tags/qe-6.3MaX -b qe-6.3MaX
DEBUG:__main__:Command stderr: Switched to a new branch 'qe-6.3MaX'

DEBUG:__main__:Command: ln -sf /Users/ashour/code/dft-tutor/work/q-e/dev-tools/input_xx.xsl /Users/ashour/code/dft-tutor/work/q-e/PW/Doc/input_xx.xsl
DEBUG:__main__:Command: /Users/ashour/code/dft-tutor/work/q-e/dev-tools/helpdoc --version 6.3 /Users/ashour/code/dft-tutor/work/q-e/PW/Doc/INPUT_PW.def
DEBUG:__main__:Command stdout: 
***
*** Parsing the helpdoc.schema
***

   parsing ROOTELEMENT input_description ... 
      parsin

In [21]:
import xmltodict
with open(os.path.join(DATABSE_DIR, '7.0', 'INPUT_PW.xml'), 'r') as f:
    doc = xmltodict.parse(f.read())

namelist = doc['input_description']['namelist'][0]
name = namelist['@name']
vars = namelist['var']
vars[5]['default']

{'ref': 'calculation',
 '#text': "1  if  == 'scf', 'nscf', 'bands';\n50 for the other cases"}

In [35]:
type_map = {
    "CHARACTER": str,
    "REAL": float,
    "INTEGER": int,
    "LOGICAL": bool,
}

new_vars = []
for v in vars:
    print(v.keys())
    name = v["@name"]
    type = type_map[v["@type"]]
    default = v.get("default", None)
    if isinstance(default, dict):
        default = default["#text"]
    options = v.get("options", None)
    if not options and type == bool:
        options = {
            True: "",
            False: "",
        }
    new_vars.append(
        {
            "name": name,
            "type": type,
            "default": default,
            "options": options,
        }
    )

# Create enum that maps the type to the corresponding python builtin type
# Map "CHARACTER" to str, REAL to float, INTEGER to int, LOGICAL to bool

dict_keys(['@name', '@type', 'default', 'options'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', 'options'])
dict_keys(['@name', '@type', 'default', 'options'])
dict_keys(['@name', '@type', 'info'])
dict_keys(['@name', '@type', 'info', 'default'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'info'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'info'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', 'options'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', 'info'])
dict_keys(['@name', '@type', 'default', '

In [36]:
new_vars

[{'name': 'calculation',
  'type': str,
  'default': "'scf'",
  'options': {'info': ['A string describing the task to be performed. Options are:',
    '(vc = variable-cell).'],
   'opt': [{'@val': "'scf'"},
    {'@val': "'nscf'"},
    {'@val': "'bands'"},
    {'@val': "'relax'"},
    {'@val': "'md'"},
    {'@val': "'vc-relax'"},
    {'@val': "'vc-md'"}]}},
 {'name': 'title', 'type': str, 'default': "' '", 'options': None},
 {'name': 'verbosity',
  'type': str,
  'default': "'low'",
  'options': {'info': ['Currently two verbosity levels are implemented:',
    {'b': ["'debug'",
      "'medium'",
      "'high';",
      "'default'",
      "'minimal'",
      "'low'"],
     '#text': 'and  have the same effect as \n and  as'}],
   'opt': [{'@val': "'high'"}, {'@val': "'low'"}]}},
 {'name': 'restart_mode',
  'type': str,
  'default': "'from_scratch'",
  'options': {'info': 'Available options are:',
   'opt': [{'@val': "'from_scratch'",
     '#text': 'From scratch. This is the normal way to per